In [32]:
import importlib
import os
from pathlib import Path

import numpy as np
import pandas as pd

import config
import module

try:
    importlib.reload(config)  # reload module
except NameError:
    pass
try:
    importlib.reload(module)  # reload module
except NameError:
    pass

In [33]:
# load df
folder_path = Path("input", config.month)

excel_files = [f for f in os.listdir(folder_path) if f.endswith('.xls')]

dfs = []

for file in excel_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path, skiprows=6)
    dfs.append(df)

df_ori = pd.concat(dfs, ignore_index=True)

WARNING *** file size (2540131) not 512 + multiple of sector size (512)
WARNING *** file size (1303139) not 512 + multiple of sector size (512)
WARNING *** file size (1614435) not 512 + multiple of sector size (512)
WARNING *** file size (2835043) not 512 + multiple of sector size (512)


# Clean DF

In [34]:
df_clean = (df_ori
    .dropna(how="all", axis="columns")
    .dropna(how="all", axis="rows")
    .rename(columns=lambda c: c.lower().replace(" ", "_"))  # replace space with _
    .assign(
        student_code=lambda df_: (
            df_["last_name"].str.upper()
            + " "
            + df_["first_name"].str.upper()
            + " - "
            + df_["student_code"].astype("str")
        ).str.strip(),
        student_membership=lambda df_: module.create_student_membership(df_),
        start_level=lambda df_: df_["start_level"].astype(float),
        current_level=lambda df_: df_["current_level"].astype(float),
        date_of_birth=lambda df_: pd.to_datetime(df_["date_of_birth"]),
        start_date=lambda df_: pd.to_datetime(df_["start_date"]),
        end_date=lambda df_: pd.to_datetime(df_["end_date"]),
        email=lambda df_: df_["email"].str.lower().str.strip(),
        mobile=lambda df_: (
            df_["mobile"]
            .astype(str)
            .str.replace("-", "", regex=False)
            .str.replace("+", "", regex=False)
            .str.strip()
        ),
    )
    # ! drop ST
    .loc[
        lambda df_: ~(
            df_["student_code"].str.contains("STREET TALK|STREETTALK", na=False)
        )
    ]
    # ! drop duplicated member based on student code and start date
    .drop_duplicates(subset=["student_code", "start_date"], keep="first")
    # ! drop unnecessary cols
    .drop(
        columns=[
            "gender",
            "home",
            "work",
            "end_level",
            "on_track",
            "course_status",
            "personal_tutor",
            "first_name",
            "last_name",
            "center_name",
        ]
    )
)
df_clean.head(3)

,student_code,date_of_birth,mobile,email,service_type,consultant,start_date,end_date,start_level,current_level,contract_status,student_membership
0,NATALIA CHRISTINE - 8816,1982-01-19 07:00:00,62083156410965,christinenata@gmail.com,Standard,WSE CAD,2022-12-20,2023-01-19 23:59:59,8.0,8.0,Promo-Invalid,Deluxe
2,(CPT PT BAYAN RESOURCES TBK) RAMCES - 9442,1982-04-28 07:00:00,6281385387510,ramces@bayan.com.sg,Standard,AIDIL MUNAWAR,2023-04-05,2023-05-16 23:59:59,15.0,15.0,Promo-Invalid,Deluxe
3,FAUZAN (CPT PT. BAYAN RESOURCES TBK) AHMAD - 9435,1994-08-27 07:00:00,6281315767413,ahmad.fauzan@bayan.com.sg,Standard,Putri Handayani Kun Andika,2023-05-23,2023-07-03 23:59:59,15.0,15.0,Promo-Invalid,Deluxe


# Assertion

In [35]:
assert sorted(df_clean["student_membership"].unique()) == ["Deluxe", "GO", "VIP"]

# Save DF

In [36]:
filename = ("coco member.xlsx").replace(" ", "_")
output_path = Path("output", folder_path.stem)

if not os.path.exists(full_filepath := output_path / filename):
    os.mkdir(output_path)
    df_clean.to_excel(full_filepath, engine="xlsxwriter", index=False)
    print("File saved.")
else:
    print("File already exist.")

File saved.
